## This is a RAG app using llamindex

In [ ]:
%pip install -q cassio datasets langchain langchain_community openai tiktoken llama-index jedi

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
!wget https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf

In [ ]:
%pip install

In [9]:

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent

In [11]:
documents = SimpleDirectoryReader("./").load_data("/content/2025-isg-outlook.pdf")

Loading files:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()



In [ ]:
query_engine.

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
def add(a: float, b: float) ->